In [ ]:
import subprocess
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import torch.nn.functional as F
from tqdm import tqdm
import re
from trl import DPOTrainer, DPOConfig, ModelConfig,get_quantization_config,get_kbit_device_map, 

# Load environment variables from /etc/network_turbo
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def strip_prompt(prompt: str, text: str) -> str:
    """
    If `text` literally starts with `prompt` (ignoring leading/trailing
    whitespace), cut that prefix off and return the remainder.
    """
    p = prompt.strip()
    # Escaping safeguards punctuation / regex metacharacters
    pattern = r"^\s*" + re.escape(p) + r"\s*"
    return re.sub(pattern, "", text, count=1).lstrip()

/root/autodl-tmp/Self_play_DRPO/self_play_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [61]:
"""
ds_ultrafeed_train = load_dataset("openbmb/UltraFeedback", split = 'train', cache_dir="/root/autodl-tmp/dataset")
ds_ultrafeed_train = ds_ultrafeed_train.filter(lambda x: len(x['completions']) > 0)
def get_preferred(sample):
    prompt = sample['instruction']
    sample_completions = sample['completions']
    scores = [sample_completions[i]['overall_score'] for i in range(len(sample_completions))]
    responses = [sample_completions[i]['response'] for i in range(len(sample_completions))]
    preferred_index = scores.index(max(scores))
    dispreferred_index = scores.index(min(scores))
    preferred_ans = responses[preferred_index]
    dispreferred_ans = responses[dispreferred_index]
    return {
        'prompt':prompt,
        'preferred': preferred_ans,
        'dispreferred': dispreferred_ans,
    }
    
processed_dataset = ds_ultrafeed_train.map(get_preferred, remove_columns=ds_ultrafeed_train.column_names)
processed_dataset.push_to_hub("august66/DRPO_data_from_ultrafeed", split="train")
"""

'\nds_ultrafeed_train = load_dataset("openbmb/UltraFeedback", split = \'train\', cache_dir="/root/autodl-tmp/dataset")\nds_ultrafeed_train = ds_ultrafeed_train.filter(lambda x: len(x[\'completions\']) > 0)\ndef get_preferred(sample):\n    prompt = sample[\'instruction\']\n    sample_completions = sample[\'completions\']\n    scores = [sample_completions[i][\'overall_score\'] for i in range(len(sample_completions))]\n    responses = [sample_completions[i][\'response\'] for i in range(len(sample_completions))]\n    preferred_index = scores.index(max(scores))\n    dispreferred_index = scores.index(min(scores))\n    preferred_ans = responses[preferred_index]\n    dispreferred_ans = responses[dispreferred_index]\n    return {\n        \'prompt\':prompt,\n        \'preferred\': preferred_ans,\n        \'dispreferred\': dispreferred_ans,\n    }\n\nprocessed_dataset = ds_ultrafeed_train.map(get_preferred, remove_columns=ds_ultrafeed_train.column_names)\nprocessed_dataset.push_to_hub("august66/

In [ ]:
device = 'cuda'
model_name = 'Qwen/Qwen2.5-0.5B-Instruct'   # use 0.5B model to test for now 
cache_path = "/root/autodl-tmp/model_cache"
model_args = ModelConfig(model_name)
model_torch_dtype = torch.float16
model_kwargs = dict(
    revision = model_args.model_revision,
    torch_dtype = model_torch_dtype, 
    trust_remote_code = model_args.trust_remote_code,
)
lm_model_instance = AutoModelForCausalLM.from_pretrained(
    model_args.model_name_or_path,
    **model_kwargs,
    cache_dir = cache_path
).eval()

lm_model_tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path, 
    padding_side = 'left', 
    use_fast = True,
    trust_remote_code = model_args.trust_remote_code,
    cache_dir = cache_path
)

if not lm_model_tokenizer.pad_token:
    lm_model_tokenizer.pad_token = lm_model_tokenizer.eos_token

print (model_torch_dtype)

seed = 42
FIRST = 20_000
SECOND = 20_000
ds_prompt_reshuffle = ds_prompt_train.shuffle(seed=seed)
ds_prompt_split_1 = ds_prompt_reshuffle.select(range(FIRST))
ds_prompt_split_2 = ds_prompt_reshuffle.select(range(FIRST, FIRST + SECOND))
ds_prompt_split_3 = ds_prompt_reshuffle.select(range(FIRST + SECOND, len(ds_prompt_reshuffle)))


def collate_fn(batch):
    batch_prompts = [row["instruction"] for row in batch]
    enc = lm_model_tokenizer(
        batch_prompts,
        padding = True,
        truncation = True,
        max_length = 512,
        return_tensors = 'pt',
    )
    return enc.input_ids, enc.attention_mask, batch_prompts

loader = DataLoader(
    ds_prompt_split_1,
    batch_size = 256,
    shuffle = False,
    collate_fn = collate_fn,
    num_workers = 4
)

lm_model_instance.to(device)
prompt_completion_list = []
for input_ids, attentio_mask, prompts in tqdm(loader):
    input_ids, attentio_mask = input_ids.to(device), attentio_mask.to(device)
    outputs = lm_model_instance.generate(
        input_ids = input_ids,
        attention_mask = attentio_mask,
        max_new_tokens = 128,
        do_sample = True,
        temperature = 1.0,
        top_p = 1.0,
        num_return_sequences = 2,
        pad_token_id = lm_model_tokenizer.pad_token_id,
        eos_token_id = lm_model_tokenizer.eos_token_id,
    )
    decoded_output = lm_model_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for i, prompt in enumerate(prompts):
        raw1, raw2 = decoded_output[2 * i : 2 * i + 2]

        # strip echoes
        resp1 = strip_prompt(prompt, raw1)
        resp2 = strip_prompt(prompt, raw2)

        prompt_completion_list.append({
            "instruction": prompt,
            "response": [resp1, resp2],
        })
ds_prompt_completion = Dataset.from_list(prompt_completion_list)


torch.float16


In [ ]:
seed = 42
FIRST = 20_000
SECOND = 20_000
ds_prompt_reshuffle = ds_prompt_train.shuffle(seed=seed)
ds_prompt_split_1 = ds_prompt_reshuffle.select(range(FIRST))
ds_prompt_split_2 = ds_prompt_reshuffle.select(range(FIRST, FIRST + SECOND))
ds_prompt_split_3 = ds_prompt_reshuffle.select(range(FIRST + SECOND, len(ds_prompt_reshuffle)))


def collate_fn(batch):
    batch_prompts = [row["instruction"] for row in batch]
    enc = lm_model_tokenizer(
        batch_prompts,
        padding = True,
        truncation = True,
        max_length = 512,
        return_tensors = 'pt',
    )
    return enc.input_ids, enc.attention_mask, batch_prompts

loader = DataLoader(
    ds_prompt_split_1,
    batch_size = 256,
    shuffle = False,
    collate_fn = collate_fn,
    num_workers = 4
)

lm_model_instance.to(device)
prompt_completion_list = []
for input_ids, attentio_mask, prompts in tqdm(loader):
    input_ids, attentio_mask = input_ids.to(device), attentio_mask.to(device)
    outputs = lm_model_instance.generate(
        input_ids = input_ids,
        attention_mask = attentio_mask,
        max_new_tokens = 128,
        do_sample = True,
        temperature = 1.0,
        top_p = 1.0,
        num_return_sequences = 2,
        pad_token_id = lm_model_tokenizer.pad_token_id,
        eos_token_id = lm_model_tokenizer.eos_token_id,
    )
    decoded_output = lm_model_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for i, prompt in enumerate(prompts):
        raw1, raw2 = decoded_output[2 * i : 2 * i + 2]

        # strip echoes
        resp1 = strip_prompt(prompt, raw1)
        resp2 = strip_prompt(prompt, raw2)

        prompt_completion_list.append({
            "instruction": prompt,
            "response": [resp1, resp2],
        })
ds_prompt_completion = Dataset.from_list(prompt_completion_list)


In [65]:
seed = 42
FIRST = 20_000
SECOND = 20_000
data_cache_path = "/root/autodl-tmp/dataset"
drpo_train = load_dataset("august66/DRPO_data_from_ultrafeed", split="train", cache_dir=data_cache_path)
drpo_train_reshuffle = drpo_train.shuffle(seed=seed)
drpo_train_split_1 = drpo_train_reshuffle.select(range(FIRST))
drpo_train_split_2 = drpo_train_reshuffle.select(range(FIRST, FIRST + SECOND))
drpo_train_split_3 = drpo_train_reshuffle.select(range(FIRST + SECOND, len(drpo_train_reshuffle)))